In [ ]:
##GEO submission update (in progress)

In [1]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import get_workflow_details, delete_wfrs
import time
import pandas as pd
import openpyxl

In [2]:
#Add key and blank GEO submission template

my_auth = get_key('default', keyfile='~/keypairs.json')  #add_key
GEO_metadata_template_file = '' #https://www.ncbi.nlm.nih.gov/geo/info/seq.htm

In [3]:
#Add datasets to be uploaded. All datasets should be from one publication or study and filter for one organism at a time.

sets_list = [] 

search_url  = ''
    esets = [ff_utils.get_metadata(i, my_auth) for i in sets_list]
elif search_url:
    esets = [i for i in ff_utils.search_metadata(search_url, my_auth)]
    
print("No. of samples {}".format(len(esets))) 

No. of samples 1


In [4]:
## Extracting STUDY SECTION 
user_input = []

for eset in esets[0:1]:
    if eset.get("produced_in_pub") == None:
        print("No publication present, add manually")
        has_pub = False
        user_input.append("title")
        user_input.append("summary")
        user_input.append("author list")
    else:
        has_pub = True
        pub_details = eset.get("produced_in_pub")
        pub_title = pub_details.get("title")
        summary = pub_details.get("abstract")
        authors = pub_details.get("authors")
        full_name = []

        for author in authors:
            name = author.split(" ")
            surname = name[0]
            if len(name) > 2:
                firstname = name[-1]
            else:
                firstname = name[1]
            author_name = firstname + ", " + surname
            full_name.append([author_name])        

No publication present, add manually


In [5]:
##Extracting SAMPLES details

##User input

#add file type
processed_files = []
supplementary_file_type = ["gene-cell expression matrix", "cell annotation", "gene annotation"]

#If you would like to add supplementary files from 4DN as processed files for GEO
add_supp_as_proc = True  
molecule = "nuclear RNA"  #add extracted molecule choices ["polyA RNA", "total RNA", "nuclear RNA", "cytoplasmic RNA", "genomic DNA", "protein", "other"]


#Don't edit below
portal_url = "https://data.4dnucleome.org/"
replicate_desc = {}
replicate_desc_short = {}
md5sums_raw = {}
md5sums_proc = {}
paired_raw_files = {}
all_raw_files = {}
all_proc_files = {}
total_supplementary_files = []
proc_file_description = {}

dataset_labels = []
conditions = []
organisms = []
all_tissues = []

tissue_name = ""
cell_line_name = ""
cell_type = ""
treatment = ""
batch = ""

samples_rows = []

for eset in esets:
    if add_supp_as_proc:
                    total_supplementary_files = []
    supplementary_files = eset.get("other_processed_files")
    for sup_file_info in supplementary_files:
        s_files = sup_file_info.get("files")
        for sfile in s_files:
            s_file_type = sfile.get("file_type")
            if s_file_type in supplementary_file_type:
                sfile_acc = sfile.get("accession")
                sfile_name = sfile.get('display_title')
                sfile_md5sum = sfile.get("md5sum")
                proc_file_description[s_file_type] = sfile_name.split(".", 1)[1]
                total_supplementary_files.append(sfile_name)
                if add_supp_as_proc:
                    md5sums_proc[sfile_name] = sfile_md5sum
    if eset.get("dataset_label") not in dataset_labels:
        dataset_label = eset.get("dataset_label")
        dataset_labels.append(dataset_label)
    condition = eset.get("condition")
    genotype = condition
    if genotype not in conditions:
        conditions.append(genotype)
    replicate_info = eset.get("replicate_exps")
    for reps in replicate_info:
            biorep = reps.get("bio_rep_no")     
            techrep = reps.get("tec_rep_no")
            rep_description = "Biological replicate " +  str(biorep) + ", Technical replicate " + str(techrep)
            rep_description_short = "B" +  str(biorep) + " T" + str(techrep)
            rep_info = reps.get("replicate_exp")
            rep_info_acc = rep_info.get("accession")
            replicate_desc[rep_info_acc] = rep_description
            replicate_desc_short[rep_info_acc] = rep_description_short
    exps = eset.get('experiments_in_set')
    for exp in exps:
        raw_files_per_exp = []
        proc_files_per_exp = []
        exp_id = exp.get("@id")
        exp_url = portal_url + exp_id
        exp_acc = exp.get("accession")
        library_name = exp.get("display_title")
        exp_details = exp.get("experiment_type")
        library_strategy = exp_details.get('display_title')
        files = exp.get("files")
        for file in files:
            file_acc = file.get("accession")
            file_type = file.get("file_type")
            file_name = file.get("display_title")
            if file_type == "reads":
                raw_files_per_exp.append(file.get('display_title'))
                file_metadata = ff_utils.get_metadata(file_acc, my_auth)
                instrument = file_metadata.get('instrument')
                md5sum = file_metadata.get('md5sum')
                md5sums_raw[file_name] = md5sum
                if file.get("paired_end"):
                    paired = "paired-end"
                    related_files = file.get("related_files")
                    for rf in related_files:
                        if rf.get("relationship_type") == "paired with":
                            paired_acc = rf.get("accession")
                            paired_raw_files[file_acc] = paired_acc
                else:
                    paired = "single"
        if exp.get("processed_files"):
            proc_files = exp.get("processed_files")
            for pfile in proc_files:
                file_acc = pfile.get("accession")
                file_type = pfile.get("file_type")
                file_name = pfile.get("display_title")
                if file_type in processed_files:
                    proc_files_per_exp.append(file_name)
                    pfile_metadata = ff_utils.get_metadata(file_acc, my_auth)
                    pmd5sum = pfile_metadata.get('md5sum')
                    md5sums_proc[file_name] = pmd5sum
        else:    
            proc_files_per_exp.append('')
        biosample = exp.get("biosample")
        biosample_id = biosample.get("@id")
        biosample_url = portal_url + biosample_id
        biosample_type = biosample.get("biosample_type")
        biosources = biosample.get("biosource")
        for biosource in biosources:
            organism = biosource.get("organism")
            organism_name = organism.get("display_title")
            organism_uuid = organism.get("uuid")
            organism_metadata = ff_utils.get_metadata(organism_uuid, my_auth)
            if organism_metadata.get("genome_assembly"):
                 organism_genome_assembly = organism_metadata.get("genome_assembly")
            else:
                 user_input.append("organism_genome_assembly")
            if organism_name not in organisms:
                organisms.append(organism_name)
            biosource_type = biosource.get("biosource_type")
            if biosource_type == "tissue":
                tissue = biosource.get('tissue')
                tissue_name = tissue.get('term_name')
                all_tissues.append(tissue_name)
            else:
                cell_type = biosource_type
                cell_line = biosource.get("cell_line")
                cell_line_name = cell_line.get("term_name")
        title =  exp_acc + ", " + organism_name + " - " + condition + ", " + replicate_desc_short[exp_acc]      
        description = library_strategy + " in " + organism_name + ", " + condition + ", " + replicate_desc[exp_acc] + ", 4DN experiment: " + exp_url + " ,4DN Biosample: " + biosample_url
        samples_rows.append([exp_acc,title,library_strategy,organism_name,tissue_name,cell_line_name, cell_type, genotype, treatment, batch, molecule, paired,instrument,description])
        all_raw_files[exp_acc] = raw_files_per_exp
        if add_supp_as_proc:
            all_proc_files[exp_acc] = total_supplementary_files
        else:    
            all_proc_files[exp_acc] = proc_files_per_exp

#Add experimental design        
genotypes = ', '.join(conditions)
experiment_design = 'Total {} samples generated using {} in {} on {}'.format(len(samples_rows),library_strategy,organism_name, genotypes)

#Add processed data files format and content
proc_descriptions = []
for file_type, ext in proc_file_description.items():
    desc = ext + ":" + file_type
    proc_descriptions.append(desc)
processed_data_files_format_content  = ', '.join(proc_descriptions)

#Warnings                                         
if len(all_proc_files) == 0:
    print("No processed files for any samples - cannot submit to GEO")
if len(organisms) > 1:
    print("Samples for more than one organisms added, please filter the search query for one organism.")
                                         
                                         
print("done")    

done


In [10]:
#mnaullay add missing data

missing = ', '.join(user_input)
print("Add the following manually: {}".format(missing))

if 'organism_genome_assembly' not in user_input:
    print("Genome assembly added {}".format(organism_genome_assembly))

Add the following manually: title, summary, author list, organism_genome_assembly, organism_genome_assembly


In [11]:
override_genome_assembly = True

if has_pub == False:
    pub_title = "Cross-species imputation and comparison of single-cell transcriptomic profiles"
    summary = "Cross-species comparison and prediction of gene expression profiles are important to understand regulatory changes during evolution and to transfer knowledge learned from model organisms to humans. Single-cell RNA-seq (scRNA-seq) profiles enable us to capture gene expression profiles with respect to variations among indi- vidual cells; however, cross-species comparison of scRNA-seq profiles is challenging because of data sparsity, batch effects, and the lack of one-to-one cell matching across species. Moreover, single-cell profiles are challenging to obtain in certain biological contexts, limiting the scope of hypothesis generation. Here we developed Icebear, a neural network framework that decomposes single-cell measurements into factors representing cell identity, species, and batch factors. Icebear enables accurate prediction of single-cell gene expression profiles across species, thereby pro- viding high-resolution cell type and disease profiles in under-characterized contexts. Icebear also facilitates direct cross-species comparison of single-cell expression profiles for conserved genes that are located on the X chromosome in eutherian mammals but on autosomes in chicken. This comparison, for the first time, revealed evolutionary and diverse adaptations of X-chromosome upregulation in mammals"
    full_name = [["Ran, Zhang"] ,["Mu, Yang"], ["Jacob, Schreiber"], ["Diana, O’Day"], ["James, Turner"], ["Jay, Shendure"], ["William, Noble"], ["Christine, Disteche"], ["Xinxian, Deng"]]

if override_genome_assembly:
    organism_genome_assembly = "ASM229v1"
    
extract_protocol = "Adult brain and heart from both male mouse and chicken were purchased from BioChemed Services, and male opossum adult brain was provided by J. Turner (MRC, UK)."
library_construction_protocol = "The data were collected by indexing cells from each species by reverse transcriptase barcoding and then processing them jointly, in which case the species identity of each cell was known based on the sequence barcode."
data_processing_step = "1. For a given sample, creating a multi-species reference genome by concatenating the reference genomes of all the species used in that sample. 2. Mapping all of the reads to the multi-species reference, retaining only reads that map uniquely using the STAR aligner with the following parameters: –out- SAMtype BAM Unsorted –outSAMmultNmax 1 –outSAMstrandField intronMotif –outFilterMultimapNmax 1. 3. Removing PCR duplicates. 4. Eliminating any read that maps to an unassembled scaffold, mitochondrial DNA, orany locus that is marked as a repeat element by RepeatMasker (http://www.repea tmasker.org). The repeat elements by RepeatMasker were retrieved from UCSC genome browser, with the exception of opossum, where we ran RepeatMasker to generate them. Repeat elements were removed using BEDtools. 5. For each cell, counting the total number of remaining reads that map to each of the three species. 6. If the sum of the second- and third-largest counts is greater than 20% of all counts, then mark the cell as a species-doublet and eliminate it. 7. Labelling the remaining cells according to their generating species."

In [12]:
#Populate STUDY, SAMPLES and PROTOCOLS section in template file

workbook = openpyxl.load_workbook(GEO_metadata_template_file)
sheet = workbook['Metadata']

#STUDY section

start_row = 39
sheet["B12"] = pub_title
sheet["B13"] = summary
sheet["B14"] = experiment_design

#author names
if len(full_name) > 7:
    sheet.insert_rows(22, amount=len(full_name)-6)
    start_row = 39 + len(full_name) - 6

    for row in range(22, 22+(len(full_name)-6)):
        sheet[f"A{row}"] = 'contributor'

for i, row_data in enumerate(full_name, start=15):
    for j, value in enumerate(row_data, start=2):
        sheet.cell(row=i, column=j, value=value)
        

#SAMPLES section

if len(samples_rows) > 16:
    add_no_rows = len(samples_rows) - 16    
    sheet.insert_rows(start_row + 16, add_no_rows + 1)
                
for i, row_data in enumerate(samples_rows, start=start_row):
    for j, value in enumerate(row_data, start=1):
        sheet.cell(row=i, column=j, value=value)

# SAMPLES section - files metadata        
edit_file_headers = False

no_of_raw_files = []
no_of_processed_files = []
raw_files = []
proc_files = []
        
for key, value in all_proc_files.items():
    no_of_processed_files.append(len(value))
    proc_files.append(value)
    
for key, value in all_raw_files.items():
    no_of_raw_files.append(len(value))
    raw_files.append(value)    
    
max_raw = max(no_of_raw_files)
max_proc = max(no_of_processed_files)

if max_proc > 2 or max_raw > 5:
    print("Updating column headers for files")
    edit_file_headers = True

if edit_file_headers:
    file_headers =  []
    for i in range(max_proc):
        file_headers.append('processed data file')
    file_headers.append('*raw file')    
    for i in range(max_raw-1):
        file_headers.append('raw file')  

    start_row_files = start_row - 1
    for col_num, value in enumerate(file_headers, start=15):
        sheet.cell(row=start_row_files, column=col_num, value=value)
        
    start_row_files = start_row
    start_column_proc = 15
    
    for i, row_data in enumerate(proc_files, start=start_row_files):
        for j, value in enumerate(row_data, start=start_column_proc):
            sheet.cell(row=i, column=j, value=value)
    
    for i, row_data in enumerate(raw_files, start=start_row_files):
        for j, value in enumerate(row_data, start=start_column_proc + max_proc):
            sheet.cell(row=i, column=j, value=value)   

else:
    start_row_files = start_row
    start_column_proc = 15
    
    for i, row_data in enumerate(proc_files, start=start_row_files):
        for j, value in enumerate(row_data, start=start_column_proc):
            sheet.cell(row=i, column=j, value=value)
    
    for i, row_data in enumerate(raw_files, start=start_row_files):
        for j, value in enumerate(row_data, start=start_column_proc + 2):
            sheet.cell(row=i, column=j, value=value)
            
            
#Populate protocol section                
for row in sheet.iter_rows():
    for cell in row:
        if cell.value == "*extract protocol":
            sheet[f"B{cell.row}"] = extract_protocol
        if cell.value == "*library construction protocol":
            sheet[f"B{cell.row}"] = library_construction_protocol
        if cell.value == "*data processing step":
            sheet[f"B{cell.row}"] = data_processing_step
        if cell.value == "*genome build/assembly":
            sheet[f"B{cell.row}"] = organism_genome_assembly
        if cell.value == "*processed data files format and content":
            sheet[f"B{cell.row}"] = processed_data_files_format_content
            
#Populate Paired-end experiment    
if len(paired_raw_files) > 0:
    for row in sheet.iter_rows():
        for cell in row:
            if cell.value == "file name 1":
                paired_files_start_row = cell.row
    
    paired_per_exp = []
    for r1, r2 in paired_raw_files.items():
        paired_per_exp.append([r1, r2])

    for i, row_data in enumerate(paired_per_exp, start=paired_files_start_row+1):
        for j, value in enumerate(row_data, start=1):
            sheet.cell(row=i, column=j, value=value)                        
workbook.save(GEO_metadata_template_file)  

print("Metadata sheet updated")

/Users/rahinavelkar/.pyenv/versions/3.8.12/envs/4dn_dcicwrangling/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Updating column headers for files


In [13]:
# Populate MD5 Checksums sheet.

add_md5sum_raw = []
add_md5sum_proc = []

for filename, md5sum in md5sums_raw.items():
    add_md5sum_raw.append([filename,md5sum ])

add_md5sum_proc = []

for filename, md5sum in md5sums_proc.items():
    add_md5sum_proc.append([filename,md5sum ])


workbook = openpyxl.load_workbook(GEO_metadata_template_file)
sheet = workbook['MD5 Checksums']

start_row = 9
for i, row_data in enumerate(add_md5sum_raw, start=start_row):
    for j, value in enumerate(row_data, start=1):
        sheet.cell(row=i, column=j, value=value)
        
for i, row_data in enumerate(add_md5sum_proc, start=start_row):
    for j, value in enumerate(row_data, start=6):
        sheet.cell(row=i, column=j, value=value)        
        
workbook.save(GEO_metadata_template_file)

print("MD5sum sheet updated")